# Lab 6 -- Decision Trees for Regression

Compared to last week, this is a very simple lab <span style="font-size:20pt;">😃</span> You'll have fun programming!

You will implement the **Classification and Regression Tree (CART)** algorithm from scratch.

The lab is broken down into the following pieces:

* Regression Criterion
* Creating Splits
* Buiding a Tree
* Making a prediction

## Exercise 1 -- Download and load the dataset

We will be using the usual Boston Housing dataset, which is available to download from ECLASS

* Download the file
* Read it and separate the target variable from the features.
* Make a 80/10/10 train/validation/test split

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [5]:
# your code here
data = pd.read_csv("BostonHousing.csv")
y = data["medv"].to_numpy()
X = data.drop("medv", axis=1).to_numpy()

The target variable will be as usual `MEDV`. Use the rest as features.

In [6]:
# your code here
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5)

## Exercise 2 -- Optimization Criterion

For regression, a simple criterion to optimize is to minimize the sum of squared errors for a given region. This is, for all datapoints in a region with size, we minimize:

$$\sum_{i=1}^N(y_i - \hat{y})^2$$

where $N$ is the number of datapoits in the region and $\hat{y}$ is the mean value of the region for the target variable. 

Implement such a function using the description below.

Please, don't use an existing implementation, refer to the [book](https://www.statlearning.com/s/ISLRSeventhPrinting.pdf), and if you need help, ask questions!

In [7]:
def regression_criterion(region):
    """
    Implements the sum of squared error criterion in a region
    
    Parameters
    ----------
    region : ndarray
        Array of shape (N,) containing the values of the target values 
        for N datapoints in the training set.
    
    Returns
    -------
    float
        The sum of squared error
        
    Note
    ----
    The error for an empty region should be infinity (use: float("inf"))
    This avoids creating empty regions
    """
    
    if len(region) == 0:
        return float("inf")
    
    mean_value = np.mean(region)
    sse = np.sum((region - mean_value) ** 2)
    
    return sse

In [8]:
# test your code
rng = np.random.default_rng(0)
print(regression_criterion(rng.random(size=40)))
print(regression_criterion(np.ones(10)))
print(regression_criterion(np.zeros(10)))
print(regression_criterion(np.array([])))

3.6200679838629544
0.0
0.0
inf


## Exercise 3 -- Make a split

In [9]:
def split_region(region, feature_index, tau):
    """
    Given a region, splits it based on the feature indicated by
    `feature_index`, the region will be split in two, where
    one side will contain all points with the feature with values 
    lower than `tau`, and the other split will contain the 
    remaining datapoints.
    
    Parameters
    ----------
    region : array of size (n_samples, n_features)
        a partition of the dataset (or the full dataset) to be split
    feature_index : int
        the index of the feature (column of the region array) used to make this partition
    tau : float
        The threshold used to make this partition
        
    Return
    ------
    left_partition : array
        indices of the datapoints in `region` where feature < `tau`
    right_partition : array
        indices of the datapoints in `region` where feature >= `tau` 
    """
    # your code here
    left_partition = np.where([region[:, feature_index] < tau])[1]
    right_partition = np.where([region[:, feature_index] >= tau])[1]

    return left_partition, right_partition

In [10]:
l, r = split_region(X_train, 2, 10)
print(l.shape, r.shape)

(224,) (180,)


## Exercise 4 -- Find the best split

The strategy is quite simple (as well as inefficient), but it helps to reinforce the concepts.
We are going to use a greedy, exhaustive algorithm to select splits, selecting the `feature_index` and the `tau` that minimizes the Regression Criterion

In [11]:
def get_split(X, y):
    """
    Given a dataset (full or partial), splits it on the feature of that minimizes the sum of squared error
    
    Parameters
    ----------
    X : array (n_samples, n_features)
        features 
    y : array (n_samples, )
        labels
    
    Returns
    -------
    decision : dictionary
        keys are:
        * 'feature_index' -> an integer that indicates the feature (column) of `X` on which the data is split
        * 'tau' -> the threshold used to make the split
        * 'left_region' -> array of indices where the `feature_index`th feature of X is lower than `tau`
        * 'right_region' -> indices not in `low_region`
    """
    # your code here
    min_sse = np.inf
    min_feature = 0
    min_tau = 0
    for each_feature in range(X.shape[1]):
        for each_tau in np.unique(X[:, each_feature]):
            left, right = split_region(X, each_feature, each_tau)
            sse1 = regression_criterion(y[left])
            sse2 = regression_criterion(y[right])
            total_sse = sse1 + sse2

            if total_sse < min_sse:
                min_sse = total_sse
                min_feature = each_feature
                min_tau = each_tau
                min_left = left
                min_right = right

    return {"feature_index":min_feature, "tau":min_tau, "left_region":min_left, "right_region":min_right}

In [27]:
node_root = get_split(X_train[:15, :], y_train[:15])
print("Node Root:")
print(node_root)

Node Root:
{'feature_index': 5, 'tau': 6.812, 'left_region': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 14], dtype=int64), 'right_region': array([12, 13], dtype=int64)}


In [26]:
node_left = get_split(X_train[node_root["left_region"], :], y_train[node_root["left_region"]])
print("Node Left 1:")
print(node_left)
node_right = get_split(X_train[node_root["right_region"], :], y_train[node_root["right_region"]])
print("Node Right 1:")
print(node_right)

Node Left 1:
{'feature_index': 6, 'tau': 59.6, 'left_region': array([ 0,  2,  3,  4,  6, 10, 12], dtype=int64), 'right_region': array([ 1,  5,  7,  8,  9, 11], dtype=int64)}
Node Right 1:
{'feature_index': 0, 'tau': 1.83377, 'left_region': array([0], dtype=int64), 'right_region': array([1], dtype=int64)}


In [36]:
node_left_left = get_split(X_train[node_root["left_region"], :][node_left["left_region"], :], y_train[node_root["left_region"]][node_left["left_region"]])
print("Node Left Left 2:")
print(node_left_left)
node_left_right = get_split(X_train[node_root["left_region"], :][node_left["right_region"], :], y_train[node_root["left_region"]][node_left["right_region"]])
print("Node Left Right 2:")
print(node_left_right)

Node Left Left 2:
{'feature_index': 5, 'tau': 6.312, 'left_region': array([0, 3, 4, 6], dtype=int64), 'right_region': array([1, 2, 5], dtype=int64)}
Node Left Right 2:
{'feature_index': 12, 'tau': 18.13, 'left_region': array([1, 3, 4, 5], dtype=int64), 'right_region': array([0, 2], dtype=int64)}


In [41]:
node_left_left_left = get_split(X_train[node_root["left_region"], :][node_left["left_region"], :][node_left_left["left_region"], :], y_train[node_root["left_region"]][node_left["left_region"]][node_left_left["left_region"]])
print("Node Left Left Left 3:")
print(node_left_left_left)

node_left_right_left = get_split(X_train[node_root["left_region"], :][node_left["right_region"], :][node_left_right["left_region"], :], y_train[node_root["left_region"]][node_left["right_region"]][node_left_right["left_region"]])
print("Node Left Right Left 3:")
print(node_left_right_left)

Node Left Left Left 3:
{'feature_index': 0, 'tau': 0.79041, 'left_region': array([0, 1, 2], dtype=int64), 'right_region': array([3], dtype=int64)}
Node Left Right Left 3:
{'feature_index': 0, 'tau': 5.69175, 'left_region': array([1, 2, 3], dtype=int64), 'right_region': array([0], dtype=int64)}


## Exercise 5 -- Recursive Splitting

The test above is an example on how to find the root node of our decision tree. The algorithm now is a greedy search until we reach a stop criterion. To find the actual root node of our decision tree, you must provide the whole training set, not just a slice of 15 rows as the test above.

The trivial stopping criterion is to recursively grow the tree until each split contains a single point (perfect node purity). If we go that far, it normally means we are overfitting.

You will implement these criteria to stop the growth:

* A node is a leaf if:
    * It has less than `min_samples` datapoints
    * It is at the `max_depth` level from the root (each split creates a new level)
    * The criterion is `0`



In [52]:
node = get_split(X_train[node_root["left_region"], :][node_left["right_region"], :], y_train[node_root["left_region"]][node_left["right_region"]])

X = X_train[node_root["left_region"], :][node_left["right_region"], :]
y = y_train[node_root["left_region"]][node_left["right_region"]]

if len(node["right_region"]) <= 3:
    node["right_region"] = np.mean(node["right_region"])
else:
    node["right_region"] = get_split(X[node["right_region"], :], y[node["right_region"]])

if len(node["left_region"]) <= 3:
    node["left_region"] = np.mean(node["left_region"])
else:
    node["left_region"] = get_split(X[node["left_region"], :], y[node["left_region"]])
node

{'feature_index': 12,
 'tau': 18.13,
 'left_region': {'feature_index': 0,
  'tau': 5.69175,
  'left_region': array([1, 2, 3], dtype=int64),
  'right_region': array([0], dtype=int64)},
 'right_region': 1.0}

In [113]:
def recursive_growth(node, min_samples, max_depth, current_depth, X, y):
    """
    Recursively grows a decision tree.
    
    Parameters
    ----------
    node : dictionary
        If the node is terminal, it contains only the "value" key, which determines the value to be used as a prediction.
        If the node is not terminal, the dictionary has the structure defined by `get_split`
    min_samples : int
        parameter for stopping criterion if a node has <= min_samples datapoints
    max_depth : int
        parameter for stopping criterion if a node belongs to this depth
    depth : int
        current distance from the root
    X : array (n_samples, n_features)
        features (full dataset)
    y : array (n_samples, )
        labels (full dataset)
    
    Notes
    -----
    To create a terminal node, a dictionary is created with a single "value" key, with a value that
    is the mean of the target variable
    
    'left' and 'right' keys are added to non-terminal nodes, which contain (possibly terminal) nodes 
    from higher levels of the tree:
    'left' corresponds to the 'left_region' key, and 'right' to the 'right_region' key
    """
    # your code here
    
    # Left Region
    # já é a o valor cerinho
    print(f"Currendt Depth: {current_depth}")
    print(node)
    print()
    if len(node["left_region"]) == 1:
        pass
    elif len(node["left_region"]) <= min_samples or current_depth >= max_depth:
        node["left"] = {"value":np.mean(y[node["left_region"]])}
        print(f"Currendt Depth: {current_depth}", "Left:", node["left"])
    else:
        X_l = X[node["left_region"], :]
        y_l = y[node["left_region"]]
        node["left_region"] = get_split(X[node["left_region"], :], y[node["left_region"]])
        recursive_growth(node["left_region"], min_samples, max_depth, current_depth + 1, X_l, y_l)

    # Right Region
    if len(node["right_region"]) == 1:
        pass
    elif len(node["right_region"]) <= min_samples or current_depth >= max_depth:
        node["right"] = {"value":np.mean(y[node["right_region"]])}
        print(f"Currendt Depth: {current_depth}", "Right:", node["right"])
    else:
        X_r = X[node["right_region"], :]
        y_r = y[node["right_region"]]
        node["right_region"] = get_split(X[node["right_region"], :], y[node["right_region"]])
        recursive_growth(node["right_region"], min_samples, max_depth, current_depth + 1, X_r, y_r)

In [111]:
k = 20
test_root = get_split(X_train[:k, :], y_train[:k])
recursive_growth(test_root, 5, 4, 1, X_train[:k, :], y_train[:k])

Currendt Depth: 1
{'feature_index': 5, 'tau': 6.812, 'left_region': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 14, 15, 16, 18, 19],
      dtype=int64), 'right_region': array([12, 13, 17], dtype=int64)}

Currendt Depth: 2
{'feature_index': 6, 'tau': 59.6, 'left_region': array([ 0,  2,  3,  4,  6, 10, 12, 14, 15], dtype=int64), 'right_region': array([ 1,  5,  7,  8,  9, 11, 13, 16], dtype=int64)}

Currendt Depth: 3
{'feature_index': 5, 'tau': 6.326, 'left_region': array([0, 2, 3, 4, 6, 7, 8], dtype=int64), 'right_region': array([1, 5], dtype=int64)}

Currendt Depth: 4
{'feature_index': 2, 'tau': 7.38, 'left_region': array([0, 2, 3], dtype=int64), 'right_region': array([1, 4, 5, 6], dtype=int64)}

Currendt Depth: 4 Left: {'value': 19.23333333333333}
Currendt Depth: 4 Right: {'value': 21.825}
Currendt Depth: 3 Right: {'value': 26.2}
Currendt Depth: 3
{'feature_index': 12, 'tau': 18.13, 'left_region': array([1, 3, 4, 5], dtype=int64), 'right_region': array([0, 2, 6, 7], dtype=in

In [109]:
# fill in the gaps with your code
min_samples = 3
max_depth = 6
root = get_split(X_train[:20, :], y_train[:20])
recursive_growth(root, min_samples, max_depth, 0, X_train[:20, :], y_train[:20])

Currendt Depth: 0
{'feature_index': 5, 'tau': 6.812, 'left_region': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 14, 15, 16, 18, 19],
      dtype=int64), 'right_region': array([12, 13, 17], dtype=int64)}

Currendt Depth: 1
{'feature_index': 6, 'tau': 59.6, 'left_region': array([ 0,  2,  3,  4,  6, 10, 12, 14, 15], dtype=int64), 'right_region': array([ 1,  5,  7,  8,  9, 11, 13, 16], dtype=int64)}

Currendt Depth: 2
{'feature_index': 5, 'tau': 6.326, 'left_region': array([0, 2, 3, 4, 6, 7, 8], dtype=int64), 'right_region': array([1, 5], dtype=int64)}

Currendt Depth: 3
{'feature_index': 2, 'tau': 7.38, 'left_region': array([0, 2, 3], dtype=int64), 'right_region': array([1, 4, 5, 6], dtype=int64)}

Left: {'value': 19.23333333333333}
Currendt Depth: 4
{'feature_index': 0, 'tau': 0.30347, 'left_region': array([2, 3], dtype=int64), 'right_region': array([0, 1], dtype=int64)}

Left: {'value': 21.1}
Right: {'value': 22.55}
Right: {'value': 26.2}
Currendt Depth: 2
{'feature_index': 1

Below we provide code to visualise the generated tree!

In [94]:
def print_tree(node, depth):
    if 'value' in node.keys():
        print('.  '*(depth-1), f"[{node['value']}]")
    else:
        print('.  '*depth, f'X_{node["feature_index"]} < {node["tau"]}')
        print_tree(node['left'], depth+1)
        print_tree(node['right'], depth+1)


In [114]:
print_tree(root, 0)

 X_5 < 6.812


KeyError: 'left'

In [64]:
print_tree(test_root, 0)

 X_4 < 0.581
.   X_10 < 15.9
.  .   X_1 < 20.0
.  .   [20.616666666666664]
.  .   [20.616666666666664]
.  .   X_0 < 0.01439
.  .   [28.200000000000003]
.  .   [28.200000000000003]
.   X_7 < 2.5671
.  .   X_0 < 9.82349
.  .   [22.175193798449612]
.  .   [22.175193798449612]
.  .   X_0 < 8.71675
.  .   [21.292307692307688]
.  .   [21.292307692307688]


# Exercise 6 -- Make a Prediction
Use the a node to predict the class of a compatible dataset

In [15]:
def predict_sample(node, sample):
    """
    Makes a prediction based on the decision tree defined by `node`
    
    Parameters
    ----------
    node : dictionary
        A node created one of the methods above
    sample : array of size (n_features,)
        a sample datapoint
    """
    # your code here
        
def predict(node, X):
    """
    Makes a prediction based on the decision tree defined by `node`
    
    Parameters
    ----------
    node : dictionary
        A node created one of the methods above
    X : array of size (n_samples, n_features)
        n_samples predictions will be made
    """
    # your code here

Now use the functions defined above to calculate the RMSE of the validation set. 
* Try first with `min_samples=20` and `max_depth=6` (for this values you should get a validation RMSE of ~8.8)

Then, experiment with different values for the stopping criteria.

In [ ]:
# your code here